<a href="https://colab.research.google.com/github/kankkw/229352-StatisticalLearning/blob/main/Lab07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Statistical Learning for Data Science 2 (229352)
#### Instructor: Donlapark Ponnoprat

#### [Course website](https://donlapark.pages.dev/229352/)

## Lab #8

There are several deep learning frameworks in Python.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/c6/PyTorch_logo_black.svg/2560px-PyTorch_logo_black.svg.png" width="100"/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<img src="https://upload.wikimedia.org/wikipedia/commons/2/2d/Tensorflow_logo.svg" width="40"/><img src="https://assets-global.website-files.com/621e749a546b7592125f38ed/62277da165ed192adba475fc_JAX.jpg" width="100"/>

In this Lab, we will use PyTorch

In [ ]:
import numpy as np

import torch

# Tensor basics

## Basic tensor creation

### Creating a scalar (1D) tensor

In [ ]:
a = torch.tensor([3.14])
print(a)
print(a.shape)

### Convert a tensor to scalar

In [ ]:
a_scalar = a.item()
print(a_scalar, type(a_scalar))

### Creating 2D tensor

In [ ]:
B = torch.tensor([[1, 2, 3],
                  [4, 5, 6]])
print(B)
print(B.shape)

## Tensor and Numpy

### Convert from tensor to numpy array

In [ ]:
B_np = B.numpy()
print(B_np, type(B_np))

### Convert from numpy array to tensor

In [ ]:
C_np = np.array([[10, 20],
                 [30, 40]])
C = torch.from_numpy(C_np)
print(C, type(C))

## PyTorch and GPU

check if GPU is available

In [ ]:
torch.cuda.is_available()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
x = torch.randn(3, 4).to(device)
x.device

In [ ]:
x_cpu = x.to("cpu")
x_cpu.device

## Basic operations

In [ ]:
u = torch.tensor([1.0, 2.0, 3.0])
v = torch.tensor([4.0, 5.0, 6.0])

print("u+v =", u + v)
print("u-v =", u - v)
print("u*v =", u * v)
print("u/v =", u / v)
print("u^2 =", u ** 2)

### Matrix multiplication

In [ ]:
M = torch.tensor([[1., 2.],
                  [3., 4.]])
N = torch.tensor([[5., 6.],
                  [7., 8.]])

print("M @ N =\n", M @ N)

### Matrix transpose

In [ ]:
print("M.T =\n", M.T)

## Creating a specific type of tensor

In [ ]:
print("zeros:\n", torch.zeros(2, 3))
print("ones:\n", torch.ones(2, 3))
print("eye:\n", torch.eye(3))
print("arange:", torch.arange(0, 10))
print("linspace:", torch.linspace(0, 1, 5))
print("rand:\n", torch.rand(2, 2))
print("randn:\n", torch.randn(2, 2))

## Tensor's shape

### Checking the shape of a tensor

In [ ]:
T = torch.randn(4, 5, 6)
print(T.shape)
print(T.size())

### Changing the shape of a tensor

In [ ]:
T2 = T.reshape(4, 30)
print(T2.shape)

In general, use `reshape`, but if you are worried about the memory usage, use `view`.

### Stacking and concatenating tensors

In [ ]:
A1 = torch.tensor([1, 2, 3])
A2 = torch.tensor([4, 5, 6])

S0 = torch.stack([A1, A2], dim=0)
S1 = torch.stack([A1, A2], dim=1)

print("stack dim=0:\n", S0)
print("stack dim=1:\n", S1)

In [ ]:
# concatenate
C0 = torch.cat([A1, A2], dim=0)
print("cat:\n", C0)

### Squeezing a tensor (removing an extra dimension)

In [ ]:
Z = torch.randn(1, 3, 1, 4)
print("before:", Z.shape)
print("after :", Z.squeeze().shape)

In [ ]:
print("squeeze dim=0:", Z.squeeze(0).shape)
print("squeeze dim=2:", Z.squeeze(2).shape)

### Unsqueezing a tensor (adding an extra dimension)

In [ ]:
H = torch.arange(6)
print("Original H:", H)
print("Original shape:", H.shape)

H1 = H.unsqueeze(0)
print("After unsqueeze(0):", H1)
print("Shape:", H1.shape)

H2 = H.unsqueeze(1)
print("After unsqueeze(1):", H2)
print("Shape:", H2.shape)


## Indexing

In [ ]:
P = torch.arange(12).reshape(3,4)
print(P)
print(P[0])
print(P[:, 0])
print(P[-1])
print(P[:, -1])
print(P[-2:])
print(P[:, -2:])

# Exercise

In this exercise, we will simulate data to perform linear regression with 200 rows and 7 variables.

1. Create three random $N(0,1)$ tensors: `X`, `b` and `e` with `X.shape = (200, 7)`, `b.shape = (8, 1)` and `e.shape = (200, 1)` respectively.
2. Create a tensor that contains only 1's with shape `(200, 1)`.
3. Modify tensor `X` by adding the tensor in 2. as the first column.
4. Compute `y` using the following formula:
$$ y = Xb + e $$.
5. Fit a linear regression to the data `X` and `y` and obtain a tensor of estimated coefficient `b_hat`. The formula for `b_hat` is given by:
$$ \hat{b} = (X^TX)^{-1}X^Ty $$
Note: use `torch.inverse(...)` to calculate the inverse
6. Compute the predictions `y_hat`, given by:
$$ \hat{y} = X\hat{b} $$
7. Convert both `y` and `y_hat` from tensor to Numpy array and calculate MSE:
$$ MSE = \frac{1}{200}\sum_{i=1}^{200} (y_i - \hat{y}_i)^2 $$

In [ ]:
X = torch.tensor([[2, 3, 2], [4, 6, 7], [7, 2, 4]])
print(X)

X = torch.tensor([[1, 2, 3, 2], [1, 4, 6, 7], [1, 7, 2, 4]])
print(X)

In [ ]:
import torch
import numpy as np

torch.manual_seed(123)

1. Create three random  N(0,1)  tensors: X, b and e with X.shape = (200, 7), b.shape = (8, 1) and e.shape = (200, 1) respectively.

In [ ]:
X = torch.randn(200, 7)
b = torch.randn(8, 1)
e = torch.randn(200, 1)

X.shape, b.shape, e.shape

2. Create a tensor that contains only 1's with shape (200, 1).

In [ ]:
ones = torch.ones(200, 1)
ones.shape

3. Modify tensor X by adding the tensor in 2. as the first column.

In [ ]:
X = torch.cat([ones, X], dim=1)
X.shape

4. Compute y using the following formula:

                                                   y = Xb+e.

In [ ]:
y = X @ b + e
y.shape

5. Fit a linear regression to the data X and y and obtain a tensor of estimated coefficient b_hat. The formula for b_hat is given by:

                                                 b^ = (XTX)−1XTy


In [ ]:
XtX = X.T @ X
XtX_inv = torch.inverse(XtX)
Xty = X.T @ y

b_hat = XtX_inv @ Xty
b_hat.shape

6. Compute the predictions y_hat, given by:
        
                                                    y^ = Xb^


In [ ]:
y_hat = X @ b_hat
y_hat.shape

7. Convert both y and y_hat from tensor to Numpy array and calculate MSE:
                           
                                          MSE = 1200∑i=1200(yi−y^i)2


In [ ]:
y_np = y.numpy()
y_hat_np = y_hat.numpy()

MSE = np.mean((y_np - y_hat_np) ** 2)
MSE